This notebook contains the code for the final project of CS 344 Artificial Intelligence at Calvin College.

It creates a number of different networks, all aimed at using the Stanford Cars dataset to identify car brands.

In [ ]:
'''
Created on 22 sty 2018
3D Object Representations for Fine-Grained Categorization
Jonathan Krause, Michael Stark, Jia Deng, Li Fei-Fei
4th IEEE Workshop on 3D Representation and Recognition, at ICCV 2013 (3dRR-13). Sydney, Australia. Dec. 8, 2013.
@author: mgdak
'''

import os
import argparse
import sys
import random
import shutil as sh
from pprint import pprint
import scipy.io as sio
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.models import Model, Sequential, model_from_json
import shutil as sh
from keras.applications.inception_v3 import InceptionV3
from keras.applications.densenet import DenseNet121
from keras import callbacks
from keras.applications.vgg19 import VGG19
from keras.applications.vgg16 import VGG16
from keras.optimizers import SGD, RMSprop
from matplotlib import pyplot as plt
from keras iport regularizers
from keras.layers.core import Flatten
from keras.layers.normalization import BatchNormalization
from keras import backend as K

train_data_dir = 'cars_train'
val_data_dir = 'cars_test'
nb_train_samples = 843
nb_val_samples = 831

Imports and Global Variable Definitions

In [ ]:
def evaluate(learning_rate, lr_decay):
    test_datagen = ImageDataGenerator(rescale=1. / 255)
    test_generator = test_datagen.flow_from_directory(
        "/home/cjw44/allCars/car_ims/cars_test",
        target_size=(224, 224),
        batch_size=16,
        class_mode='categorical')

    # load json and create model
    json_file = open("./carRecognition_finalModel" + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("./carRecognition_finalModel" + '.h5')
    print('Loaded model from disk')

    # evaluate loaded model on test data
    sgd = SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)
    loaded_model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    score = loaded_model.evaluate_generator(test_generator, 3957 / 16, workers=6)
    print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1] * 100))

# Creates an array with following values: picture name, picture category ID, train/validation label       
def readData(matFile):
    content = sio.loadmat(matFile)
    data = [(_[0][0][:], _[5][0][0], _[6][0][0]) for _ in content['annotations'][0]]
    return data


# Creates an array of all classes
def readClasses(matFile):
    content = sio.loadmat(matFile)
    classes = [(_[0]) for _ in content['class_names'][0]]
    return classes


# Movces raw data (pictures) into respective category subfolders with train/validation division 
def dataPreprocessing(dataDir, labelsFile):
    data = readData(labelsFile)
    classes = readClasses(labelsFile)
    print("---------------")
    for recData in data:
        if recData[2] == 1:
            # validation set
            os.makedirs(dataDir + "/" + val_data_dir + "/" + classes[recData[1] - 1] + "/", exist_ok=True)
            sh.move(dataDir + "/" + recData[0][8:],
                    dataDir + "/" + val_data_dir + "/" + classes[recData[1] - 1] + "/" + recData[0][8:])
        else:
            os.makedirs(dataDir + "/" + train_data_dir + "/" + classes[recData[1] - 1] + "/", exist_ok=True)
            sh.move(dataDir + "/" + recData[0][8:],
                    dataDir + "/" + train_data_dir + "/" + classes[recData[1] - 1] + "/" + recData[0][8:])  # train set


# serializes the trained model and its weights
def serializeModel(model, fileName):
    # serialize model to JSON
    model_json = model.to_json()
    with open(fileName + ".json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights(fileName + ".h5")
    print("Saved model to disk")


def prepareDataGenerators(batchSize, srcImagesDir, labelsFile):
    classes = readClasses(labelsFile)
    # this is the augmentation configuration used for training
    train_datagen = ImageDataGenerator(
        rescale=1. / 255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
    # this is the augmentation configuration used for testing:
    # only rescaling
    test_datagen = ImageDataGenerator(rescale=1. / 255)
    # this is a generator that will read pictures found in
    # subfolers of 'car_ims_dir/train', and indefinitely generate
    # batches of augmented image data
    train_generator = train_datagen.flow_from_directory(
        srcImagesDir + "/" + train_data_dir + "/",  # this is the target directory
        target_size=(224, 224),  # all images will be resized to 299x299
        batch_size=batchSize,
        class_mode='categorical')  # since we use categorical_crossentropy loss, we need categorical labels
    # this is a similar generator, for validation data
    validation_generator = test_datagen.flow_from_directory(
        srcImagesDir + "/" + val_data_dir + "/",
        target_size=(224, 224),
        batch_size=batchSize,
        class_mode='categorical')
    return classes, train_generator, validation_generator

Define functions that allow for training and using neural networks